**What you learn:**

In this notebook you will learn about automatic unit tests in Python.   

Jens Dittrich, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

This notebook is available on https://github.com/BigDataAnalyticsGroup/python.

## unittests

How do we make sure that a certain function does what we expect it to do?

The answer is **unittests**. A unittest checks whether a given function produces an expected output for a given input.

So, for any Python function out = f(in) that we implement, a unittest defines typically multiple (in, out)-tuples.

In any **real** software delevopment unittests are the state-of-the-art method to control the correctness of your software. Unittests are typically executed automatically (every night, every time you change anything, etc.)

Do not confuse the terms `try out` with `testing`.

`try out`: let's run my program with a couple of inputs and see what happens, if it looks good, we are done. Really?

`testing`: write unittests for each and every situation we expect our function to do. Run these tests systematically every time you change anything in your code. This is also called `automatic testing`. If we pass all tests, we are done.



#### Principle structure of a test

In [1]:
import unittest

# 'class' wraps a number of tests into a single unit
# this comes from a programming paradigm called "object-oriented programming" (OOP),
# which you do not really need for this lecture
# simply read it as "a class bundles a couple of test-functions into one unit"
#
# in OOP-terms:
# "this is a class definition extending/inheriting from unittest.TestCase" 
class MyTest(unittest.TestCase):

    # the following function will be called before every test-function:
    def setUp(self):
        print('setUp MyTest')
        
    # each test-function must have a name starting with 'test':
    def test_1(self):
        print('test_1')

    def test_2(self):
        print('test_2')

    def test_3(self):
        print('test_3')

    # the following function will be called after every test-function:
    def tearDown(self):
        print('tearDown')

In [2]:
# Run the unit test without shutting down the jupyter kernel
# notice that this call will look for all test classes, i.e.
# all classes inheriting from unittest.TestCase
# each of these functions will be executed once
# unittest.main(argv=['ignored', '-v'], verbosity=2, exit=False)

# only execute a specific Test-class:
unittest.main(argv=['ignored', '-v'], defaultTest='MyTest', verbosity=2, exit=False)


test_1 (__main__.MyTest) ... ok
test_2 (__main__.MyTest) ... ok
test_3 (__main__.MyTest) ... 

setUp MyTest
test_1
tearDown
setUp MyTest
test_2
tearDown
setUp MyTest
test_3
tearDown


ok

----------------------------------------------------------------------
Ran 3 tests in 0.007s

OK


#### Example Testing Scenario

In [22]:
# some class we implemented:
class Bar:
    def __init__(self):
        self.a = 42

    def whatever(self, par):
        if par == 0:
            raise ValueError("Division by zero is not defined.")
        self.a /= par
        return self.a

In [4]:
b = Bar()

In [5]:
b.a

42

In [8]:
type(b.whatever(2))

float

In [23]:
# unittests in Python are very similar to unittests in Java:
import unittest

class BarTest(unittest.TestCase):
    # the following method will be called before every test-Method
    def setUp(self):
        print("setup BarTest")
        self.bar = Bar()
        
    def test_div(self):
        print('test_div')
        self.assertEqual(self.bar.a, 42)
        local_a = 42
        par = 77
        ret = self.bar.whatever(par)
        self.assertEqual(ret, local_a/par)
        self.assertEqual(self.bar.a, local_a/par)
        
    def test_init(self):
        print('test_init')
        self.assertEqual(self.bar.a, 42)

    def test_divbyzero(self):
        print('test_divbyzero')
        with self.assertRaises(ValueError):
            self.bar.whatever(0)

In [24]:
# only execute a specific Test-class:
unittest.main(argv=['ignored', '-v'], defaultTest='BarTest', verbosity=2, exit=False)

test_div (__main__.BarTest) ... ok
test_divbyzero (__main__.BarTest) ... ok
test_init (__main__.BarTest) ... 

setup BarTest
test_div
setup BarTest
test_divbyzero
setup BarTest
test_init


ok

----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


#### Example: sum of squares

We want to compute $$ \sum_{i=low}^{high} i^2$$

In [25]:
# returns the sum of squares in the int interval [low;high]
# a straightforward implementation of a squaredSum
def mySquaredSum(low, high):
    _sum = 0
    for i in range(low,high+1): # note the 'high+1' (rather than 'high')
        _sum += i*i

    return _sum

In [31]:
# let's "try it out":
mySquaredSum(1,10)

385

In [32]:
# let's "try it out" even more:
mySquaredSum(12,11)

0

In [33]:
# let's "try it out" even more:
mySquaredSum(0,-12)

0

Looks good. But are we done here? No, we need to write proper unittests for this.

#### Unittest for sum of squares

OK, let's write a real unittest for our squaredSum-function:

In [40]:
# returns the sum of squares in the int interval [low;high]
def squaredSumRecursive(low, high, count=0, indent=''):
    #print(indent, 'sqsr', low, high)
    if count > 15: # recursion emergency brake...
        return 0
    _sum = 0
    if high>low:
        middleLeft = low + (high-low)//2
        middleRight = middleLeft + 1
        _sum += squaredSumRecursive(low, middleLeft, count+1, indent+'  ')
        _sum += squaredSumRecursive(middleRight, high, count+1, indent+'  ')
        return _sum
    elif high<low:
        return 0
    else:
        return high**2

1,2

3,4,5

6,7,8,

9,10

In [41]:
squaredSumRecursive(1,10)

385

An alternative implementation of squared sums:

In [48]:
import random as rand
import unittest

# 'class' wraps a number of tests into a single unit
class SumTest(unittest.TestCase):

    # test one specific call to squaredSum:
    def test_Simple(self):
        self.assertEqual(mySquaredSum(1,10), 385)

    # test multiple specific calls to squaredSum:
    def test_EmptyInterval(self):
        self.assertEqual(mySquaredSum(12,11), 0)
        self.assertEqual(mySquaredSum(0,-12), 0)
        
    # test several random calls to squaredSum:
    def test_randomRanges(self):
        rand.seed(42)
        for i in range(100000):
            low = rand.randrange(1,50)
            high = low + rand.randrange(-3,10)
            #print(low,high)
            # the following hardly makes sense if we copy-pasted the code for mySquaredSum from squaredSum above:
            # self.assertEqual(squaredSum(low, high), mySquaredSum(low, high))
            # so actually we need an alternate implementation of squaredSum...
            # print(low,high)
            self.assertEqual(mySquaredSum(low, high), squaredSumRecursive(low, high))
            # or using a list comprehension:
            self.assertEqual(mySquaredSum(low, high), sum([i*i for i in range(low,high+1)]))
            

In [49]:
# Run the unit test without shutting down the jupyter kernel
# here we are running only SumTest!
unittest.main(argv=['ignored', '-v'], defaultTest='SumTest', verbosity=2, exit=False)

test_EmptyInterval (__main__.SumTest) ... ok
test_Simple (__main__.SumTest) ... ok
test_randomRanges (__main__.SumTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.137s

OK
